In [1]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import ttest_1samp
import numpy as np
import psycopg2
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#opening connection & cursor
conn = psycopg2.connect(
                            host="lhl-data-bootcamp.crzjul5qln0e.ca-central-1.rds.amazonaws.com",
                            database="mid_term_project",
                            user="lhl_student",
                            password="lhl_student")
cursor = conn.cursor()

def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

In [3]:
df = pd.DataFrame()

for i in range(1, 13):
    if i == 1:
        test2 = f"SELECT * FROM flights WHERE arr_delay IS NOT NULL AND weather_delay IS NOT NULL AND arr_delay < 200 AND EXTRACT(month from CAST(fl_date as date)) = {i} AND EXTRACT(year from CAST(fl_date as date)) = 2019 LIMIT 100000;"
        df = create_pandas_table(test2)
    else:
        test2 = f"SELECT * FROM flights WHERE arr_delay IS NOT NULL AND weather_delay IS NOT NULL AND arr_delay < 200 AND EXTRACT(month from CAST(fl_date as date)) = {i} AND EXTRACT(year from CAST(fl_date as date)) = 2019 LIMIT 100000;"
        test_pd_table = create_pandas_table(test2)
        df = pd.concat([df, test_pd_table]) 

In [4]:
passengers = create_pandas_table("SELECT * FROM passengers TABLESAMPLE SYSTEM(10) WHERE year >= 2019 ")

In [5]:
passengers.shape

(48380, 38)

In [9]:
passengers.passengers.max()

75124.0

In [7]:
fuel = create_pandas_table("SELECT * FROM fuel_comsumption WHERE year >= 2019 ")

In [8]:
fuel.shape
fuel

,month,airline_id,unique_carrier,carrier,carrier_name,carrier_group_new,sdomt_gallons,satl_gallons,spac_gallons,slat_gallons,...,sdomt_cost,satl_cost,spac_cost,slat_cost,sint_cost,ts_cost,tdomt_cost,tint_cost,total_cost,year
0,1,21161.0,09Q,09Q,"Swift Air, LLC d/b/a Eastern Air Lines d/b/a E...",2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1831212.0,1618745.0,3449957.0,2019
1,1,21352.0,0WQ,0WQ,Avjet Corporation,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019
2,1,21492.0,1BQ,1BQ,"Eastern Airlines f/k/a Dynamic Airways, LLC",1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,102795.0,74193.0,176988.0,2019
3,1,21652.0,27Q,27Q,"Jet Aviation Flight Services, Inc.",1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019
4,1,21712.0,2HQ,2HQ,Elite Airways LLC,1,66815.0,0.0,0.0,0.0,...,175499.0,0.0,0.0,0.0,0.0,175499.0,1518919.0,0.0,1518919.0,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,9,20377.0,X9,X9,Omni Air International LLC,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,808310.0,8797931.0,9606241.0,2019
454,9,20207.0,XP,XP,XTRA Airways,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019
455,9,20378.0,YV,YV,Mesa Airlines Inc.,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019
456,9,20452.0,YX,YX,Republic Airline,3,22895.0,0.0,0.0,0.0,...,46818.0,0.0,0.0,0.0,0.0,46818.0,46818.0,0.0,46818.0,2019


In [41]:
fuel_mean=fuel[['unique_carrier','total_gallons']].groupby(['unique_carrier'],as_index=False).mean()

In [42]:
fuel_mean.rename(columns={'unique_carrier':'op_unique_carrier'},inplace=True)


In [43]:
passengers['capacity'] = passengers['seats'] - passengers['passengers']


In [44]:
passengers.rename(columns={'unique_carrier':'mkt_unique_carrier'},inplace=True)


In [45]:
passengers_mean=passengers[['mkt_unique_carrier','origin','dest','passengers', 'capacity']].groupby(['mkt_unique_carrier','origin','dest', ],as_index=False).mean()

In [46]:
passengers_mean.head()

,mkt_unique_carrier,origin,dest,passengers,capacity
0,04Q,BED,BTV,7.0,1.0
1,04Q,BED,FOK,2.0,6.0
2,04Q,BED,HPN,1.0,7.0
3,04Q,BED,JFK,1.0,7.0
4,04Q,BED,ME3,30.0,10.0


In [55]:
df_merged=pd.merge(df,passengers_mean,on=['mkt_unique_carrier', 'origin','dest'], how='inner') 

In [56]:
df_merged=pd.merge(df_merged,fuel_mean, on=['op_unique_carrier'], how='inner') 

In [57]:
df_merged.shape

(807078, 45)

In [58]:
df_merged.isna().sum()

fl_date                     0
mkt_unique_carrier          0
branded_code_share          0
mkt_carrier                 0
mkt_carrier_fl_num          0
op_unique_carrier           0
tail_num                    0
op_carrier_fl_num           0
origin_airport_id           0
origin                      0
origin_city_name            0
dest_airport_id             0
dest                        0
dest_city_name              0
crs_dep_time                0
dep_time                    0
dep_delay                   0
taxi_out                    0
wheels_off                  0
wheels_on                   0
taxi_in                     0
crs_arr_time                0
arr_time                    0
arr_delay                   0
cancelled                   0
cancellation_code      807078
diverted                    0
dup                         0
crs_elapsed_time            1
actual_elapsed_time         0
air_time                    0
flights                     0
distance                    0
carrier_de

In [51]:
# remember to close connection to allow for others to use the bandwidth
cursor.close()
conn.close()

In [52]:
df_merged.passengers.unique()

array([   nan,  128. , 5422. , ...,  371.5,  443. , 1336. ])

In [26]:
df.mkt_unique_carrier.unique()

array(['AA', 'NK', 'UA', 'DL', 'WN', 'AS', 'G4', 'HA', 'B6', 'F9'],
      dtype=object)

In [27]:
df.shape

(120000, 42)

In [28]:
df_merged.shape

(83508, 45)

In [29]:
df_merged.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,passengers,capacity,total_gallons
0,2019-01-01,NK,NK,NK,811,NK,N640NK,811,13204,MCO,...,14.0,0.0,0.0,None,NaN,NaN,None,4849.75,416.25,3.937194e+07
1,2019-01-01,NK,NK,NK,803,NK,N625NK,803,13204,MCO,...,1.0,0.0,66.0,None,NaN,NaN,None,4849.75,416.25,3.937194e+07
2,2019-01-02,NK,NK,NK,811,NK,N646NK,811,13204,MCO,...,15.0,0.0,0.0,None,NaN,NaN,None,4849.75,416.25,3.937194e+07
3,2019-03-01,NK,NK,NK,811,NK,N693NK,811,13204,MCO,...,16.0,0.0,0.0,None,NaN,NaN,None,4849.75,416.25,3.937194e+07
4,2019-04-01,NK,NK,NK,801,NK,N671NK,801,13204,MCO,...,1.0,0.0,58.0,None,NaN,NaN,None,4849.75,416.25,3.937194e+07


In [54]:
df_test = df_merged[['mkt_unique_carrier','origin','dest','dep_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay', 'weather_delay', 'carrier_delay']].groupby(['mkt_unique_carrier','origin','dest', ],as_index=False).mean()

In [ ]:
df_merged.drop(columns=['first_dep_time','total_add_gtime','longest_add_gtime', 'no_name']

In [60]:
df_merged = pd.merge(df_merged, df_test, on=['mkt_unique_carrier', 'origin', 'dest'] )

In [63]:
df_merged = df_merged.drop(columns=['first_dep_time','total_add_gtime','longest_add_gtime', 'no_name'])

In [69]:
df_merged = df_merged.drop(columns=['dep_delay_x', 'carrier_delay_x', 'weather_delay_x', 'nas_delay_x', 'security_delay_x',
       'late_aircraft_delay_x',])

In [74]:
len(df_merged.columns)

41

In [72]:
df_merged.to_csv('flights_merged.csv')